In [5]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
import missingno
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score

from matplotlib import font_manager, rc
# font_path = "dataset/malgun.ttf"
# font_name = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font_name)

In [129]:
# 가을14, 15년도 데이터 전체에 고객마다 라벨값을 추가한 데이터 read
df = pd.read_pickle('dataset/full_data_with_label.pkl')

In [130]:
df

,RECEIPTNO,CUSTNO,PURCHDATE,PURCHTIME,AFFIL,CLASSCODEL,CLASSCODEM,CLASSCODES,STORENO,GENDER,AGEGROUP,RESIDENCE,PURCHAMOUNT,Year,AFFIL_CLASS,spent_less
0,08068631,07122,2014-09-06,17,B,8,0802,B080203,028,M,40세~44세,100,14000,2014,B8,1.0
1,08942410,17967,2014-11-30,15,B,15,1502,B150203,050,M,20세~24세,100,1750,2014,B15,0.0
2,08942410,17967,2014-11-30,15,B,47,4703,B470302,050,M,20세~24세,100,7900,2014,B47,0.0
3,08942410,17967,2014-11-30,15,B,77,7707,B770702,050,M,20세~24세,100,16898,2014,B77,0.0
4,08942410,17967,2014-11-30,15,B,82,8202,B820201,050,M,20세~24세,100,7731,2014,B82,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449415,07143341,08054,2015-10-20,18,B,45,4506,B450601,013,F,35세~39세,055,15800,2015,B45,0.0
3449416,07143342,05446,2015-10-20,19,B,18,1807,B180707,013,F,45세~49세,055,2000,2015,B18,1.0
3449417,07598192,01584,2015-09-16,10,B,90,9001,B900103,018,F,55세~59세,100,11850,2015,B90,1.0
3449418,07598193,00033,2015-09-16,15,B,5,0510,B051005,018,M,60세이상,100,3160,2015,B5,1.0


In [131]:
# CUSTNO별 총 매출액 
a = df.groupby('CUSTNO').sum()['PURCHAMOUNT'].sort_values(ascending = False)

print (a)

# 사분위수 확인
a.describe()[0:7].round(2)


CUSTNO
14599    1358712770
15999     816659700
08994     677330720
16406     594581200
17129     419075490
            ...    
17965          9330
16387          6890
18604          4500
14370          3900
08301           990
Name: PURCHAMOUNT, Length: 19349, dtype: int64


count       19349.00
mean      9518088.29
std      20136516.18
min           990.00
25%       2652851.00
50%       4331337.00
75%      10091512.00
Name: PURCHAMOUNT, dtype: float64

In [135]:
# 데이터프레임화 후 vip변수 추가
df = pd.DataFrame(a)
# 고객별 총매출액기준 3사분위 이상의 매출액을 갖는 고객들을 vip, 아니면 'normal'

def vip (x):
    if x >= 11753719 : return 'vip'
    else : return 'normal'

df['Vip'] = df.PURCHAMOUNT.apply(vip)

df

,PURCHAMOUNT,Vip
CUSTNO,,
14599,1358712770,vip
15999,816659700,vip
08994,677330720,vip
16406,594581200,vip
17129,419075490,vip
...,...,...
17965,9330,normal
16387,6890,normal
18604,4500,normal


In [137]:
# null값 없음
df.isnull().sum()

PURCHAMOUNT    0
Vip            0
dtype: int64

In [133]:
# normal과 vip 고객수 - 3사분위로 나눠서 vip가 25%정도
df.Vip.value_counts()


normal    15114
vip        4235
Name: Vip, dtype: int64

In [70]:
a.describe()[0:7].round(0)

count       16773.0
mean     10432885.0
std      20502690.0
min         19900.0
25%       2977878.0
50%       4944355.0
75%      11753719.0
Name: PURCHAMOUNT, dtype: float64

In [72]:
b.describe()

count    16773.000000
mean        15.613720
std          0.960219
min          9.898525
25%         14.906722
50%         15.413757
75%         16.279680
max         21.029804
Name: PURCHAMOUNT, dtype: float64